In [1]:
# from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
# from pyspark.ml.classification import LogisticRegression
# from pyspark.sql import SparkSession

# # Membuat session Spark
# spark = SparkSession.builder.appName("OneHotExample").getOrCreate()

# # Data contoh
# data = spark.createDataFrame([
#     ("Indonesia", 1),
#     ("USA", 0),
#     ("India", 1),
#     ("Canada", 0),
#     ("Brazil", 1)
# ], ["negara", "label"])

# # Langkah 1: StringIndexer untuk kolom 'negara'
# indexer = StringIndexer(inputCol="negara", outputCol="negara_index")
# indexed_data = indexer.fit(data).transform(data)

# # Langkah 2: OneHotEncoder untuk kolom yang sudah diindeks
# encoder = OneHotEncoder(inputCol="negara_index", outputCol="negara_onehot")
# encoded_data = encoder.fit(indexed_data).transform(indexed_data)

# # Langkah 3: Gabungkan fitur (menggunakan VectorAssembler untuk membuat vektor fitur)
# assembler = VectorAssembler(inputCols=["negara_onehot"], outputCol="features")
# assembled_data = assembler.transform(encoded_data)

# # Langkah 4: Melatih model Logistic Regression
# lr = LogisticRegression(featuresCol="features", labelCol="label")
# model = lr.fit(assembled_data)

# # Menampilkan hasil model
# print("Model telah dilatih dengan sukses!")


In [2]:
# import findspark
# findspark.init()
# findspark.find()
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler, MinMaxScaler, OneHotEncoder, StringIndexer

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier

from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
spark = SparkSession.builder.appName("Classification with Spark").getOrCreate()

In [4]:
help(spark.read.csv)

Help on method csv in module pyspark.sql.readwriter:

csv(path: Union[str, List[str]], schema: Union[pyspark.sql.types.StructType, str, NoneType] = None, sep: Optional[str] = None, encoding: Optional[str] = None, quote: Optional[str] = None, escape: Optional[str] = None, comment: Optional[str] = None, header: Union[bool, str, NoneType] = None, inferSchema: Union[bool, str, NoneType] = None, ignoreLeadingWhiteSpace: Union[bool, str, NoneType] = None, ignoreTrailingWhiteSpace: Union[bool, str, NoneType] = None, nullValue: Optional[str] = None, nanValue: Optional[str] = None, positiveInf: Optional[str] = None, negativeInf: Optional[str] = None, dateFormat: Optional[str] = None, timestampFormat: Optional[str] = None, maxColumns: Union[str, int, NoneType] = None, maxCharsPerColumn: Union[str, int, NoneType] = None, maxMalformedLogPerPartition: Union[str, int, NoneType] = None, mode: Optional[str] = None, columnNameOfCorruptRecord: Optional[str] = None, multiLine: Union[bool, str, NoneType] 

In [5]:
dataset = spark.read.csv("diabetes.csv",header=True, inferSchema=True)

In [6]:
dataset.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [7]:
dataset.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [8]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [9]:
new_data.printSchema()

root
 |-- Pregnancies: float (nullable = true)
 |-- Glucose: float (nullable = true)
 |-- BloodPressure: float (nullable = true)
 |-- SkinThickness: float (nullable = true)
 |-- Insulin: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- DiabetesPedigreeFunction: float (nullable = true)
 |-- Age: float (nullable = true)
 |-- Outcome: float (nullable = true)



In [10]:
new_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction| Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+
|        6.0|  148.0|         72.0|         35.0|    0.0|33.6|                   0.627|50.0|    1.0|
|        1.0|   85.0|         66.0|         29.0|    0.0|26.6|                   0.351|31.0|    0.0|
|        8.0|  183.0|         64.0|          0.0|    0.0|23.3|                   0.672|32.0|    1.0|
|        1.0|   89.0|         66.0|         23.0|   94.0|28.1|                   0.167|21.0|    0.0|
|        0.0|  137.0|         40.0|         35.0|  168.0|43.1|                   2.288|33.0|    1.0|
|        5.0|  116.0|         74.0|          0.0|    0.0|25.6|                   0.201|30.0|    0.0|
|        3.0|   78.0|         50.0|         32.0|   88.0|31.0|                   0.248|26.0

In [11]:
from pyspark.sql.functions import col, count, isnan, when
#checking for null ir nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [12]:
cols=new_data.columns
cols.remove("Outcome")
assembler = VectorAssembler(inputCols=cols,outputCol="features")

# Now let us use the transform method to transform our dataset
data=assembler.transform(new_data)

data.select("features",'Outcome').show(truncate=False)

+-----------------------------------------------------------------------+-------+
|features                                                               |Outcome|
+-----------------------------------------------------------------------+-------+
|[6.0,148.0,72.0,35.0,0.0,33.599998474121094,0.6269999742507935,50.0]   |1.0    |
|[1.0,85.0,66.0,29.0,0.0,26.600000381469727,0.35100001096725464,31.0]   |0.0    |
|[8.0,183.0,64.0,0.0,0.0,23.299999237060547,0.671999990940094,32.0]     |1.0    |
|[1.0,89.0,66.0,23.0,94.0,28.100000381469727,0.16699999570846558,21.0]  |0.0    |
|[0.0,137.0,40.0,35.0,168.0,43.099998474121094,2.2880001068115234,33.0] |1.0    |
|[5.0,116.0,74.0,0.0,0.0,25.600000381469727,0.20100000500679016,30.0]   |0.0    |
|[3.0,78.0,50.0,32.0,88.0,31.0,0.24799999594688416,26.0]                |1.0    |
|[10.0,115.0,0.0,0.0,0.0,35.29999923706055,0.1340000033378601,29.0]     |0.0    |
|[2.0,197.0,70.0,45.0,543.0,30.5,0.15800000727176666,53.0]              |1.0    |
|[8.0,125.0,96.0

In [13]:
standardscaler=StandardScaler()
sc = standardscaler.setInputCol("features").setOutputCol("Scaled_features")
data_scaled=sc.fit(data).transform(data)

In [14]:
data_scaled.select("features",'Outcome','Scaled_features').show(truncate=False)

+-----------------------------------------------------------------------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                               |Outcome|Scaled_features                                                                                                                                          |
+-----------------------------------------------------------------------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|[6.0,148.0,72.0,35.0,0.0,33.599998474121094,0.6269999742507935,50.0]   |1.0    |[1.7806383732194306,4.628960915766174,3.7198138711154307,2.1940523222807116,0.0,4.261709202425419,1.8923810993699686,4.251616970894646]                  |
|[1.0,85.0,66.0,29.0,0.0,26.600000381469727,0.3510000109

In [15]:
assembled_data = data_scaled.select("Scaled_features","Outcome")
assembled_data.show()

+--------------------+-------+
|     Scaled_features|Outcome|
+--------------------+-------+
|[1.78063837321943...|    1.0|
|[0.29677306220323...|    0.0|
|[2.37418449762590...|    1.0|
|[0.29677306220323...|    0.0|
|[0.0,4.2849165233...|    1.0|
|[1.48386531101619...|    0.0|
|[0.89031918660971...|    1.0|
|[2.96773062203238...|    0.0|
|[0.59354612440647...|    1.0|
|[2.37418449762590...|    1.0|
|[1.18709224881295...|    0.0|
|[2.96773062203238...|    1.0|
|[2.96773062203238...|    0.0|
|[0.29677306220323...|    1.0|
|[1.48386531101619...|    1.0|
|[2.07741143542266...|    1.0|
|[0.0,3.6906580274...|    1.0|
|[2.07741143542266...|    1.0|
|[0.29677306220323...|    0.0|
|[0.29677306220323...|    1.0|
+--------------------+-------+
only showing top 20 rows



In [16]:
train, test = assembled_data.randomSplit([0.8, 0.2])

In [17]:
dist=dict(train.select("Outcome").rdd.countByValue()).items()
for k,v in dist:
    print(k[0],v/train.count()*100)
print("==========================================")
dist=dict(test.select("Outcome").rdd.countByValue()).items()
for k,v in dist:
    print(k[0],v/test.count()*100)

0.0 65.58966074313409
1.0 34.41033925686591
0.0 63.08724832214765
1.0 36.91275167785235


---

## Stratified Split

In [18]:
assembled_data.count()

768

In [19]:
print(assembled_data.select("Outcome").distinct().rdd.collect())
print(assembled_data.select("Outcome").distinct().rdd.collect()[0])
print(assembled_data.select("Outcome").distinct().rdd.collect()[0][0])

[Row(Outcome=1.0), Row(Outcome=0.0)]
Row(Outcome=1.0)
1.0


In [20]:
fractions = assembled_data.select("Outcome").distinct().rdd.map(lambda row: (row[0], 0.7)).collect()
fractions = dict(fractions)
print(fractions)

train = assembled_data.sampleBy("Outcome",fractions, seed=42)
test = assembled_data.subtract(train)

print("Train Data Count: ", train.count())
print("Test Data Count: ", test.count())
print("Total Data Count: ", test.count() + train.count())

{1.0: 0.7, 0.0: 0.7}
Train Data Count:  569
Test Data Count:  199
Total Data Count:  768


In [21]:
train_distribution = train.groupBy("Outcome").count()
train_distribution = train_distribution.withColumn("percentage", col("count")/train.count()*100)
train_distribution.show()

test_distribution = test.groupBy("Outcome").count()
test_distribution = test_distribution.withColumn("percentage", col("count")/test.count()*100)
test_distribution.show()

+-------+-----+-----------------+
|Outcome|count|       percentage|
+-------+-----+-----------------+
|    1.0|  195|34.27065026362039|
|    0.0|  374|65.72934973637962|
+-------+-----+-----------------+

+-------+-----+-----------------+
|Outcome|count|       percentage|
+-------+-----+-----------------+
|    1.0|   73|36.68341708542713|
|    0.0|  126|63.31658291457286|
+-------+-----+-----------------+



---

In [22]:
train.count(),train.show()

+--------------------+-------+
|     Scaled_features|Outcome|
+--------------------+-------+
|[1.78063837321943...|    1.0|
|[0.29677306220323...|    0.0|
|[0.29677306220323...|    0.0|
|[0.0,4.2849165233...|    1.0|
|[1.48386531101619...|    0.0|
|[2.96773062203238...|    0.0|
|[1.18709224881295...|    0.0|
|[2.96773062203238...|    1.0|
|[2.96773062203238...|    0.0|
|[0.0,3.6906580274...|    1.0|
|[2.07741143542266...|    1.0|
|[0.29677306220323...|    0.0|
|[0.89031918660971...|    0.0|
|[2.07741143542266...|    1.0|
|[2.96773062203238...|    1.0|
|[2.07741143542266...|    1.0|
|[0.29677306220323...|    0.0|
|[0.89031918660971...|    1.0|
|[1.78063837321943...|    0.0|
|[3.26450368423562...|    0.0|
+--------------------+-------+
only showing top 20 rows



(569, None)

In [23]:
test.count(),test.show()

+--------------------+-------+
|     Scaled_features|Outcome|
+--------------------+-------+
|[1.78063837321943...|    0.0|
|[2.07741143542266...|    1.0|
|[0.89031918660971...|    0.0|
|[0.29677306220323...|    0.0|
|[0.0,5.6298173299...|    1.0|
|[2.67095755982914...|    1.0|
|[3.56127674643886...|    1.0|
|[0.29677306220323...|    0.0|
|[2.07741143542266...|    0.0|
|[2.96773062203238...|    0.0|
|[1.18709224881295...|    0.0|
|[1.18709224881295...|    0.0|
|[1.48386531101619...|    1.0|
|[0.29677306220323...|    0.0|
|[1.48386531101619...|    0.0|
|[0.89031918660971...|    0.0|
|[2.96773062203238...|    0.0|
|[0.59354612440647...|    0.0|
|[0.0,3.1589530573...|    0.0|
|[0.89031918660971...|    1.0|
+--------------------+-------+
only showing top 20 rows



(199, None)

## Logistic Regression

In [24]:
# help(LogisticRegression)

In [25]:
log_reg = LogisticRegression(labelCol="Outcome", featuresCol="Scaled_features",maxIter=40) # default maxIter=100
model=log_reg.fit(train)

In [26]:
# log_reg.getOrDefault("maxIter")

In [27]:
prediction_test=model.transform(test)

In [28]:
prediction_test.show()

+--------------------+-------+--------------------+--------------------+----------+
|     Scaled_features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.78063837321943...|    0.0|[0.67900848767766...|[0.66351736621234...|       0.0|
|[2.07741143542266...|    1.0|[0.91160181629810...|[0.71332783204551...|       0.0|
|[0.89031918660971...|    0.0|[1.41249242044969...|[0.80415876615470...|       0.0|
|[0.29677306220323...|    0.0|[2.77515687416964...|[0.94131849102158...|       0.0|
|[0.0,5.6298173299...|    1.0|[-2.7963699132256...|[0.05752065446034...|       1.0|
|[2.67095755982914...|    1.0|[-0.4758345061456...|[0.38323622720395...|       1.0|
|[3.56127674643886...|    1.0|[1.25497516102834...|[0.77815989707538...|       0.0|
|[0.29677306220323...|    0.0|[-0.3847829506785...|[0.40497382743567...|       1.0|
|[2.07741143542266...|    0.0|[1.03363191179512...|[0.73761941157160...|    

In [29]:
prediction_test.select("Outcome","prediction").show(10)

+-------+----------+
|Outcome|prediction|
+-------+----------+
|    0.0|       0.0|
|    1.0|       0.0|
|    0.0|       0.0|
|    0.0|       0.0|
|    1.0|       1.0|
|    1.0|       1.0|
|    1.0|       0.0|
|    0.0|       1.0|
|    0.0|       0.0|
|    0.0|       0.0|
+-------+----------+
only showing top 10 rows



---

In [30]:
q_pred = test.select("Scaled_features")
q_pred.show()

+--------------------+
|     Scaled_features|
+--------------------+
|[1.78063837321943...|
|[2.07741143542266...|
|[0.89031918660971...|
|[0.29677306220323...|
|[0.0,5.6298173299...|
|[2.67095755982914...|
|[3.56127674643886...|
|[0.29677306220323...|
|[2.07741143542266...|
|[2.96773062203238...|
|[1.18709224881295...|
|[1.18709224881295...|
|[1.48386531101619...|
|[0.29677306220323...|
|[1.48386531101619...|
|[0.89031918660971...|
|[2.96773062203238...|
|[0.59354612440647...|
|[0.0,3.1589530573...|
|[0.89031918660971...|
+--------------------+
only showing top 20 rows



In [31]:
pred = model.transform(q_pred)
pred.show()

+--------------------+--------------------+--------------------+----------+
|     Scaled_features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[1.78063837321943...|[0.67900848767766...|[0.66351736621234...|       0.0|
|[2.07741143542266...|[0.91160181629810...|[0.71332783204551...|       0.0|
|[0.89031918660971...|[1.41249242044969...|[0.80415876615470...|       0.0|
|[0.29677306220323...|[2.77515687416964...|[0.94131849102158...|       0.0|
|[0.0,5.6298173299...|[-2.7963699132256...|[0.05752065446034...|       1.0|
|[2.67095755982914...|[-0.4758345061456...|[0.38323622720395...|       1.0|
|[3.56127674643886...|[1.25497516102834...|[0.77815989707538...|       0.0|
|[0.29677306220323...|[-0.3847829506785...|[0.40497382743567...|       1.0|
|[2.07741143542266...|[1.03363191179512...|[0.73761941157160...|       0.0|
|[2.96773062203238...|[0.70331813536116...|[0.66892303449076...|       0.0|
|[1.18709224

---

In [32]:
# Compute raw scores on the test set
predictionAndLabels = prediction_test.select("Outcome","prediction").rdd

In [33]:
predictionAndLabels.collect()

[Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=1.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=1.0, prediction=1.0),
 Row(Outcome=1.0, prediction=1.0),
 Row(Outcome=1.0, prediction=0.0),
 Row(Outcome=0.0, prediction=1.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=1.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=1.0, prediction=1.0),
 Row(Outcome=0.0, prediction=1.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=1.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=0.0, prediction=0.0),
 Row(Outcome=1.0, prediction=0.0),
 Row(Outcome=0.0, pr

In [34]:
help(MulticlassClassificationEvaluator)

Help on class MulticlassClassificationEvaluator in module pyspark.ml.evaluation:

class MulticlassClassificationEvaluator(JavaEvaluator, pyspark.ml.param.shared.HasLabelCol, pyspark.ml.param.shared.HasPredictionCol, pyspark.ml.param.shared.HasWeightCol, pyspark.ml.param.shared.HasProbabilityCol, pyspark.ml.util.JavaMLReadable, pyspark.ml.util.JavaMLWritable)
 |  MulticlassClassificationEvaluator(*, predictionCol: str = 'prediction', labelCol: str = 'label', metricName: 'MulticlassClassificationEvaluatorMetricType' = 'f1', weightCol: Optional[str] = None, metricLabel: float = 0.0, beta: float = 1.0, probabilityCol: str = 'probability', eps: float = 1e-15)
 |  
 |  Evaluator for Multiclass Classification, which expects input
 |  columns: prediction, label, weight (optional) and probabilityCol (only for logLoss).
 |  
 |  .. versionadded:: 1.5.0
 |  
 |  Examples
 |  --------
 |  >>> scoreAndLabels = [(0.0, 0.0), (0.0, 1.0), (0.0, 0.0),
 |  ...     (1.0, 0.0), (1.0, 1.0), (1.0, 1.0), (1.0

In [35]:
evaluator = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="accuracy")
accuracy_lr = evaluator.evaluate(prediction_test)
print(f"Accuracy: {accuracy_lr}")

# Evaluasi precision, recall, dan F1-score
precision = evaluator.setMetricName("weightedPrecision").evaluate(prediction_test)
recall = evaluator.setMetricName("weightedRecall").evaluate(prediction_test)
f1_score = evaluator.setMetricName("f1").evaluate(prediction_test)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.7537688442211056
Precision: 0.7490750455574577
Recall: 0.7537688442211055
F1 Score: 0.7453248490804213


In [36]:
# help(BinaryClassificationMetrics)

In [37]:
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)
print("Area under PR = %s" % metrics.areaUnderPR)

c:\Users\micha\Anaconda3\envs\PySparkEnv\lib\site-packages\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Area under ROC = 0.7417582417582418
Area under PR = 0.509868324007513


## NaiveBayes

In [38]:
# help(NaiveBayes)

In [39]:
naive_bayes = NaiveBayes(featuresCol='Scaled_features',labelCol='Outcome',smoothing=1.0)

In [40]:
model = naive_bayes.fit(train)

In [41]:
# select example rows to display.
prediction_test = model.transform(test)

In [42]:
prediction_test.show()

+--------------------+-------+--------------------+--------------------+----------+
|     Scaled_features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.78063837321943...|    0.0|[-46.361374930926...|[0.63156663200021...|       0.0|
|[2.07741143542266...|    1.0|[-52.930266416654...|[0.54933129458851...|       0.0|
|[0.89031918660971...|    0.0|[-31.153240407360...|[0.64732025283577...|       0.0|
|[0.29677306220323...|    0.0|[-27.915121723352...|[0.70519574803775...|       0.0|
|[0.0,5.6298173299...|    1.0|[-48.412500439396...|[0.73541714453796...|       0.0|
|[2.67095755982914...|    1.0|[-39.567752893829...|[0.51964275877738...|       0.0|
|[3.56127674643886...|    1.0|[-48.999990469351...|[0.44190439767670...|       1.0|
|[0.29677306220323...|    0.0|[-42.796681776404...|[0.57791194290637...|       0.0|
|[2.07741143542266...|    0.0|[-41.839411641841...|[0.61396175206550...|    

In [43]:
prediction_test.select("Outcome","prediction").show(10)

+-------+----------+
|Outcome|prediction|
+-------+----------+
|    0.0|       0.0|
|    1.0|       0.0|
|    0.0|       0.0|
|    0.0|       0.0|
|    1.0|       0.0|
|    1.0|       0.0|
|    1.0|       1.0|
|    0.0|       0.0|
|    0.0|       0.0|
|    0.0|       0.0|
+-------+----------+
only showing top 10 rows



In [44]:
predictionAndLabels = prediction_test.select("Outcome","prediction").rdd

In [45]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="accuracy")
accuracy_nb = evaluator.evaluate(prediction_test)
print(f"Accuracy: {accuracy_nb}")

# Evaluasi precision, recall, dan F1-score
precision = evaluator.setMetricName("weightedPrecision").evaluate(prediction_test)
recall = evaluator.setMetricName("weightedRecall").evaluate(prediction_test)
f1_score = evaluator.setMetricName("f1").evaluate(prediction_test)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.6381909547738693
Precision: 0.6152413854989232
Recall: 0.6381909547738693


In [46]:
# help(BinaryClassificationMetrics)

In [47]:
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)
print("Area under PR = %s" % metrics.areaUnderPR)

c:\Users\micha\Anaconda3\envs\PySparkEnv\lib\site-packages\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Area under ROC = 0.6060267857142857
Area under PR = 0.050590526015596576


In [48]:
dir(metrics)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_java_model',
 '_sc',
 'areaUnderPR',
 'areaUnderROC',
 'call',
 'unpersist']

## GBTClassifier

In [49]:
# help(GBTClassifier)

In [50]:
gradient_boost_class = GBTClassifier(labelCol="Outcome", featuresCol="Scaled_features", maxIter=15, maxDepth=3)

In [51]:
model = gradient_boost_class.fit(train)

In [52]:
prediction_test = model.transform(test)

In [53]:
prediction_test.show()

+--------------------+-------+--------------------+--------------------+----------+
|     Scaled_features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.78063837321943...|    0.0|[0.15900642016410...|[0.57883988991847...|       0.0|
|[2.07741143542266...|    1.0|[0.11851246867129...|[0.55898036211403...|       0.0|
|[0.89031918660971...|    0.0|[1.10388238936473...|[0.90094462675756...|       0.0|
|[0.29677306220323...|    0.0|[1.44477519839414...|[0.94732745170723...|       0.0|
|[0.0,5.6298173299...|    1.0|[-1.0177412580408...|[0.11552753159056...|       1.0|
|[2.67095755982914...|    1.0|[-0.5770369130842...|[0.23974576892522...|       1.0|
|[3.56127674643886...|    1.0|[-0.2085155765135...|[0.39722738694457...|       1.0|
|[0.29677306220323...|    0.0|[-0.8119539910943...|[0.16466661660882...|       1.0|
|[2.07741143542266...|    0.0|[0.61485937313149...|[0.77376934449642...|    

In [54]:
prediction_test.select("Outcome","prediction").show(10)

+-------+----------+
|Outcome|prediction|
+-------+----------+
|    0.0|       0.0|
|    1.0|       0.0|
|    0.0|       0.0|
|    0.0|       0.0|
|    1.0|       1.0|
|    1.0|       1.0|
|    1.0|       1.0|
|    0.0|       1.0|
|    0.0|       0.0|
|    0.0|       0.0|
+-------+----------+
only showing top 10 rows



In [55]:
predictionAndLabels = prediction_test.select("Outcome","prediction").rdd

In [56]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="accuracy")
accuracy_gbt = evaluator.evaluate(prediction_test)
print(f"Accuracy: {accuracy_gbt}")

# Evaluasi precision, recall, dan F1-score
precision = evaluator.setMetricName("weightedPrecision").evaluate(prediction_test)
recall = evaluator.setMetricName("weightedRecall").evaluate(prediction_test)
f1_score = evaluator.setMetricName("f1").evaluate(prediction_test)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.7587939698492462
Precision: 0.7587939698492463
Recall: 0.7587939698492463


In [57]:
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)
print("Area under PR = %s" % metrics.areaUnderPR)

Area under ROC = 0.7403783431180692
Area under PR = 0.621194733283607


## RandomForestClassifier

In [58]:
# help(RandomForestClassifier)

In [112]:
random_forest_classifier = RandomForestClassifier(labelCol="Outcome", featuresCol="Scaled_features", numTrees=50, maxDepth=10)

In [113]:
model = random_forest_classifier.fit(train)

In [114]:
prediction_test = model.transform(test)

In [115]:
prediction_test.show()

+--------------------+-------+--------------------+--------------------+----------+
|     Scaled_features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.78063837321943...|    0.0|[28.2431135531135...|[0.56486227106227...|       0.0|
|[2.07741143542266...|    1.0|[19.2615384615384...|[0.38523076923076...|       1.0|
|[0.89031918660971...|    0.0|[41.7308603613215...|[0.83461720722643...|       0.0|
|[0.29677306220323...|    0.0|          [50.0,0.0]|           [1.0,0.0]|       0.0|
|[0.0,5.6298173299...|    1.0|[18.9333333333333...|[0.37866666666666...|       1.0|
|[2.67095755982914...|    1.0|[19.7982298301694...|[0.39596459660338...|       1.0|
|[3.56127674643886...|    1.0|[17.9333333333333...|[0.35866666666666...|       1.0|
|[0.29677306220323...|    0.0|         [19.0,31.0]|         [0.38,0.62]|       1.0|
|[2.07741143542266...|    0.0|[36.2509259259259...|[0.72501851851851...|    

In [116]:
prediction_test.select("Outcome","prediction").show(10)

+-------+----------+
|Outcome|prediction|
+-------+----------+
|    0.0|       0.0|
|    1.0|       1.0|
|    0.0|       0.0|
|    0.0|       0.0|
|    1.0|       1.0|
|    1.0|       1.0|
|    1.0|       1.0|
|    0.0|       1.0|
|    0.0|       0.0|
|    0.0|       0.0|
+-------+----------+
only showing top 10 rows



In [117]:
predictionAndLabels = prediction_test.select("Outcome","prediction").rdd

In [118]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="accuracy")
accuracy_rf = evaluator.evaluate(prediction_test)
print(f"Accuracy: {accuracy_rf}")

# Evaluasi precision, recall, dan F1-score
precision = evaluator.setMetricName("weightedPrecision").evaluate(prediction_test)
recall = evaluator.setMetricName("weightedRecall").evaluate(prediction_test)
f1_score = evaluator.setMetricName("f1").evaluate(prediction_test)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.7537688442211056
Precision: 0.7545016976775771
Recall: 0.7537688442211055


In [119]:
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)
print("Area under PR = %s" % metrics.areaUnderPR)

Area under ROC = 0.7350810810810812
Area under PR = 0.620663015187005


---

In [120]:
acc_dict = {"GBT":accuracy_gbt,"LR":accuracy_lr,"NB":accuracy_nb,"RF":accuracy_rf}
acc_dict = sorted(acc_dict.items(), key=lambda x: x[1], reverse=True)
for k,v in acc_dict:
    print(f"Accuracy of {k} : {v}")

# print("Accuracy of GBT : ",accuracy_gbt)
# print("Accuracy of LR : ",accuracy_lr)
# print("Accuracy of NB : ",accuracy_nb)
# print("Accuracy of RF : ",accuracy_rf)

Accuracy of GBT : 0.7587939698492462
Accuracy of LR : 0.7537688442211056
Accuracy of RF : 0.7537688442211056
Accuracy of NB : 0.6381909547738693
